In [1]:
import geopandas as gpd
import osmnx as ox
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.animation import FuncAnimation

import city2graph as c2g

In [2]:
# Download and project the street network to British National Grid (EPSG:27700) for metric distances
# Data should be obtained from Only in Soho, London
G_drive = ox.graph_from_place(
    "Teresina, Piauí",
    network_type="all",
)
G_proj = ox.project_graph(G_drive, to_crs="EPSG:31983")

# Convert to dual graph
street_primary_nodes, street_primary_edges = c2g.nx_to_gdf(G_proj)
projected_crs = street_primary_edges.crs

In [4]:
amenity_tags = ["clinic","hospital"]
amenity_candidates = ox.features_from_place(
    "Teresina, Piauí",
    tags={"amenity": amenity_tags},
).to_crs(projected_crs)

print(f"Street primary edges: {len(street_primary_edges):,}")
print(f"Street primary nodes: {len(street_primary_nodes):,}")
print(f"Candidate Amenities: {len(amenity_candidates):,}")

Street primary edges: 94,018
Street primary nodes: 34,540
Candidate Amenities: 298


In [5]:
# Convert to dual graph
street_dual_nodes, street_dual_edges = c2g.dual_graph((street_primary_nodes, street_primary_edges))

# Drop the multi-index
street_dual_nodes = street_dual_nodes.droplevel(2)

In [6]:
street_dual_nodes.head()

,,osmid,highway,name,oneway,ref,reversed,length,geometry,weight,maxspeed,...,lanes,access,service,bridge,width,area,tunnel,mm_len,index_position,index
743182.943735,9.432727e+06,29152940,trunk,Avenida Getúlio Vargas,True,BR-316,False,85.726693,"LINESTRING (743140.281 9432722.522, 743184.973...",None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.844401,0.0,"(320563363, 7180675058, 0)"
743213.435739,9.432726e+06,46262062,primary,Avenida Presidente Getulio Vargas,True,NaN,False,27.669921,"LINESTRING (743201.384 9432719.204, 743225.488...",None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.665606,29760.0,"(2324660988, 7180675058, 0)"
743241.867326,9.432739e+06,29152940,trunk,Avenida Getúlio Vargas,True,BR-316,False,34.717278,"LINESTRING (743225.488 9432732.783, 743258.247...",None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.742267,1.0,"(7180675058, 7237891186, 0)"
743162.533802,9.432733e+06,29153266,trunk,Avenida Getúlio Vargas,True,BR-316,False,49.274600,"LINESTRING (743184.786 9432743.697, 743140.281...",None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.285554,42.0,"(320668480, 320563363, 0)"
743317.304693,9.432752e+06,775847037,service,NaN,False,NaN,True,124.104911,"LINESTRING (743373.904 9432775.251, 743363.848...",None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,124.178166,85156.0,"(7237891180, 7237891186, 0)"


In [12]:
# Name street dual nodes
street_dual_nodes.index.name = "segment_id"

# Add distance to dual edges
street_dual_edges["distance_m"] = street_dual_edges.geometry.length

# Collapse complex Amenity geometries to points within the projected CRS
amenities = (
    amenity_candidates[["name", "amenity", "geometry"]]
    .copy()
    .explode(index_parts=False)
    .dropna(subset=["geometry"])
)
non_point_mask = ~amenities.geometry.geom_type.isin(["Point"])
amenities.loc[non_point_mask, "geometry"] = amenities.loc[non_point_mask, "geometry"].centroid
amenities = amenities.set_geometry("geometry")
amenities["name"] = amenities["name"].fillna(amenities["amenity"].str.title())
amenities = amenities[~amenities.geometry.is_empty]
amenities = amenities.drop_duplicates(subset="geometry").reset_index(drop=True)

In [13]:
amenities.head()

,name,amenity,geometry
0,UltraClinica,clinic,POINT (743070.235 9436885.063)
1,CCDT-Centro Clínico de Diagnóstico de Teresina,clinic,POINT (742626.267 9437448.796)
2,ProntoOftalmo,clinic,POINT (743135.602 9436865.792)
3,Hospital São Marcos,hospital,POINT (743633.943 9436955.89)
4,UBS São Camilo,clinic,POINT (745500.414 9432357.934)


In [15]:
len(amenities.name)

299